In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from loess.loess_1d import loess_1d

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-DJF"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [6]:
db.list_allowed_coverage_identifiers(
    session, 
    coverage_configuration_id=cov.configuration.id, 
    select_configuration_parameter_values=[
        db.get_configuration_parameter_value_by_names(session, "scenario", "rcp26"),
        db.get_configuration_parameter_value_by_names(session, "time_period", "MAM"),
    ]
)

['tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-MAM',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-JJA',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-SON']

In [7]:
db.list_coverage_identifiers(
    session,
    matching_configuration_parameters=[
        db.get_configuration_parameter_value_by_names(session, "scenario", "rcp26"),
        db.get_configuration_parameter_value_by_names(session, "climatological_model", "model_ensemble"),
    ]
)

([CoverageInternal(configuration=CoverageConfiguration(netcdf_main_dataset_name='cdd', observation_variable_aggregation_type=None, thredds_url_pattern='ensembletwbc/std/clipped/eca_cdd_an_avgagree_{time_window}_{scenario}_{year_period}_ls_VFVGTAA.nc', uncertainty_lower_bounds_coverage_configuration_id=None, name='cdd_30yr_anomaly_annual_agree_model_ensemble', wms_main_layer_name='consecutive_dry_days_index_per_time_period-uncertainty_group', uncertainty_upper_bounds_coverage_configuration_id=None, id=UUID('d50ef8e7-1a84-435e-84e6-294664b0964c'), unit='gg', display_name_english=None, palette='uncert-stippled/div-BrBG-inv', display_name_italian=None, color_scale_min=-40.0, description_english=None, color_scale_max=40.0, description_italian=None, observation_variable_id=None, coverage_id_pattern='{name}-{scenario}-{time_window}-{year_period}'), identifier='cdd_30yr_anomaly_annual_agree_model_ensemble-rcp26-tw1-DJF'),
  CoverageInternal(configuration=CoverageConfiguration(netcdf_main_datas

In [3]:
cov.configuration.coverage_id_pattern

'{name}-{climatological_model}-{measure}-{scenario}-{year_period}'

In [4]:
cov.configuration.retrieve_configuration_parameters(cov.identifier)

{'climatological_model': 'model_ensemble',
 'measure': 'absolute',
 'scenario': 'rcp26',
 'year_period': 'DJF'}

In [5]:
cov.configuration.retrieve_used_values(cov.identifier)

[ConfigurationParameterPossibleValue(configuration_parameter_value_id=UUID('c5381cd2-ca57-48a9-a096-71568e812366'), coverage_configuration_id=UUID('06c104d2-a502-400b-9728-b8e2d9ee65d9')),
 ConfigurationParameterPossibleValue(configuration_parameter_value_id=UUID('88fa9565-8af6-4ab5-afeb-bfa00cd97955'), coverage_configuration_id=UUID('06c104d2-a502-400b-9728-b8e2d9ee65d9')),
 ConfigurationParameterPossibleValue(configuration_parameter_value_id=UUID('70c82437-5e90-4181-96c7-262a097d18ba'), coverage_configuration_id=UUID('06c104d2-a502-400b-9728-b8e2d9ee65d9')),
 ConfigurationParameterPossibleValue(configuration_parameter_value_id=UUID('4842fa35-b13a-446d-ad9f-43a45b82bd23'), coverage_configuration_id=UUID('06c104d2-a502-400b-9728-b8e2d9ee65d9'))]

In [7]:
cov.configuration.get_thredds_url_fragment(cov.identifier)

'ensymbc/clipped/tas_avg_rcp26_DJF_ts19762100_ls_VFVG.nc'

In [6]:
cov.configuration.build_coverage_identifier(
    parameters=[
        db.get_configuration_parameter_value_by_names(session, "climatological_model", "model_ensemble"),
        db.get_configuration_parameter_value_by_names(session, "measure", "absolute"),
        db.get_configuration_parameter_value_by_names(session, "scenario", "rcp26"),
        db.get_configuration_parameter_value_by_names(session, "year_period", "DJF"),
    ]
)

'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-DJF'

In [7]:
cov.configuration.get_seasonal_aggregation_query_filter(coverage_identifier)

<Season.WINTER: 'WINTER'>

In [9]:
db.list_allowed_coverage_identifiers(session, coverage_configuration_id=cov.configuration.id)

['tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-MAM',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-JJA',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-SON',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp45-DJF',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp45-MAM',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp45-JJA',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp45-SON',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp85-DJF',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp85-MAM',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp85-JJA',
 'tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp85-SON']